In [2]:
!pip install transformers datasets evaluate rouge_score accelerate

In [3]:

import transformers

transformers.__version__

'4.44.2'

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from huggingface_hub import list_datasets

datasets_list = list_datasets()

num = 0
for dataset in datasets_list:
  print(dataset)

  if (num > 10):
    break

  num += 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetInfo(id='amirveyseh/acronym_identification', author='amirveyseh', sha='15ef643450d589d5883e289ffadeb03563e80a9e', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 39, 57, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=925, downloads_all_time=None, likes=19, paperswithcode_id='acronym-identification', tags=['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'source_datasets:original', 'language:en', 'license:mit', 'size_categories:10K<n<100K', 'format:parquet', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'arxiv:2010.14678', 'region:us', 'acronym-identification'], card_data=None, siblings=None)
DatasetInfo(id='ade-benchmark-corpus/ade_corpus_v2', author='ade-benchmark-corpus', sha='4ba01c71687dd7c996597042449448ea312126cf', 

In [6]:
from datasets import load_dataset

ds = load_dataset("nbeerbower/gutenberg2-dpo")

ds

README.md:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

gb2_2024_9_20.json:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/293 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['book', 'chapter', 'chosen', 'summary', 'prompt', 'rejected'],
        num_rows: 293
    })
})

In [7]:
ds = load_dataset("nbeerbower/gutenberg2-dpo", split = "train[:3%]")

ds

Dataset({
    features: ['book', 'chapter', 'chosen', 'summary', 'prompt', 'rejected'],
    num_rows: 9
})

In [8]:
ds.shape

(9, 6)

In [9]:
(ds.description)


''

In [10]:
ds[0]

{'book': 'Alice in Wonderland',
 'chapter': 1,
 'chosen': 'Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought Alice “without pictures or conversations?” So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her. There was nothing so _very_ remarkable in that; nor did Alice think it so _very_ much out of the way to hear the Rabbit say to itself, “Oh dear! Oh dear! I shall be late!” (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually _to

In [11]:
ds = ds.train_test_split(test_size = 0.2, seed = 42)

ds

DatasetDict({
    train: Dataset({
        features: ['book', 'chapter', 'chosen', 'summary', 'prompt', 'rejected'],
        num_rows: 7
    })
    test: Dataset({
        features: ['book', 'chapter', 'chosen', 'summary', 'prompt', 'rejected'],
        num_rows: 2
    })
})

In [13]:
def clean_txt(text):
    text = text.lower()
    text = text.replace("/", "")
    text = text.replace("\n", "")
    return text

cleaned_ds = ds.map(lambda example: {'summary': clean_txt(example['summary'])})


In [14]:
ds["train"]["summary"][3]


"Alice finds herself at a peculiar tea party with the March Hare, the Hatter, and a sleeping Dormouse. The Hatter asks Alice a riddle about a raven and a writing desk, but he does not know the answer either. The Hatter and the March Hare discuss the nature of time, and the Hatter explains that he quarrelled with Time last March, which is why it is always six o'clock for him. Alice learns about three little sisters living at the bottom of a treacle well, and she tries to understand how they drew treacle from the well. However, the conversation becomes confusing and rude, so Alice decides to leave the tea party. She discovers a door in a tree and enters it, finding herself back in the long hall with the little glass table. This time, she successfully unlocks the door to the garden and finds herself in a beautiful garden with bright flower-beds and cool fountains."

In [15]:
cleaned_ds["train"]["summary"][3]



"alice finds herself at a peculiar tea party with the march hare, the hatter, and a sleeping dormouse. the hatter asks alice a riddle about a raven and a writing desk, but he does not know the answer either. the hatter and the march hare discuss the nature of time, and the hatter explains that he quarrelled with time last march, which is why it is always six o'clock for him. alice learns about three little sisters living at the bottom of a treacle well, and she tries to understand how they drew treacle from the well. however, the conversation becomes confusing and rude, so alice decides to leave the tea party. she discovers a door in a tree and enters it, finding herself back in the long hall with the little glass table. this time, she successfully unlocks the door to the garden and finds herself in a beautiful garden with bright flower-beds and cool fountains."

In [16]:
EXAMPLE_TEXT_INDEX = 1

example_text = cleaned_ds["test"]["summary"][EXAMPLE_TEXT_INDEX]

example_text

'alice, feeling tired from sitting with her sister, sees a white rabbit who checks his pocket watch, sparking her curiosity. she follows the rabbit down a deep rabbit-hole that leads to a long fall through a well filled with books and maps. as she falls, she wonders if she\'ll reach the earth\'s center or go right through it. after the long fall, alice finds herself in a long hall with locked doors. she discovers a small door leading to a beautiful garden, but she is too large to fit through it. on a nearby table, she finds a bottle labeled "drink me." upon drinking it, alice shrinks to ten inches tall, the perfect size to fit through the door. however, she realizes she forgot the golden key and, after trying to reach it, sits down and cries. she then finds a cake with the words "eat me" and decides to eat it, hoping it will either make her grow larger to reach the key or smaller to fit under the door. the cake doesn\'t affect her size, leaving her frustrated that life isn\'t as out-of

In [17]:
MODEL_NAME = "sshleifer/distilbart-cnn-12-6"

In [18]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline("summarization", model = MODEL_NAME)

summary_txt = summarizer(prefix + example_text)

summary_txt

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'summary_text': ' Alice follows a rabbit down a rabbit-hole that leads to a long fall through a well filled with books and maps . She discovers a small door leading to a beautiful garden, but she is too large to fit through it . She then finds a cake with the words "eat me" and decides to eat it, hoping it will make her grow larger to reach the key .'}]

In [19]:
summarizer = pipeline("summarization", model = MODEL_NAME, truncation=True)


summary_txt = summarizer(prefix + example_text)

summary_txt

[{'summary_text': ' Alice follows a rabbit down a rabbit-hole that leads to a long fall through a well filled with books and maps . She discovers a small door leading to a beautiful garden, but she is too large to fit through it . She then finds a cake with the words "eat me" and decides to eat it, hoping it will make her grow larger to reach the key .'}]

In [20]:
ref_txt = cleaned_ds["test"]["chapter"][EXAMPLE_TEXT_INDEX]

ref_txt

1

In [21]:
print(ds.column_names)

{'train': ['book', 'chapter', 'chosen', 'summary', 'prompt', 'rejected'], 'test': ['book', 'chapter', 'chosen', 'summary', 'prompt', 'rejected']}


In [22]:
import evaluate

rouge = evaluate.load("rouge")

rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [23]:
reference = "the elephant was found near the river and everyone was glad"

ex_1 = "the big elephant was found near the turbulent river and everyone was ecstatic"

ex_2 = "the big elephant was found near the river and made everyone glad"


In [24]:
results = rouge.compute(predictions = [ex_1], references = [reference])

results

{'rouge1': 0.8333333333333333,
 'rouge2': 0.6363636363636365,
 'rougeL': 0.8333333333333333,
 'rougeLsum': 0.8333333333333333}

In [25]:
results = rouge.compute(predictions = [ex_2], references = [reference])

results

{'rouge1': 0.8695652173913043,
 'rouge2': 0.5714285714285713,
 'rougeL': 0.8695652173913043,
 'rougeLsum': 0.8695652173913043}

In [26]:
summary_txt[0]["summary_text"]

' Alice follows a rabbit down a rabbit-hole that leads to a long fall through a well filled with books and maps . She discovers a small door leading to a beautiful garden, but she is too large to fit through it . She then finds a cake with the words "eat me" and decides to eat it, hoping it will make her grow larger to reach the key .'

In [27]:
article_texts = cleaned_ds["test"]["chosen"]

article_summaries = cleaned_ds["test"]["summary"]

In [28]:

from tqdm import tqdm

candidate_summaries = []

prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(prefix + text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 2/2 [00:40<00:00, 20.24s/it]


In [29]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.23575360809403362,
 'rouge2': 0.04959758551307847,
 'rougeL': 0.14434855924217627,
 'rougeLsum': 0.14434855924217627}

In [30]:
result_unagg = rouge.compute(predictions = candidate_summaries,
                             references = article_summaries[:50],
                             use_stemmer = True,
                             use_aggregator = False)
result_unagg


{'rouge1': [0.2127659574468085, 0.25874125874125875],
 'rouge2': [0.04285714285714286, 0.056338028169014086],
 'rougeL': [0.14184397163120566, 0.14685314685314685],
 'rougeLsum': [0.14184397163120566, 0.14685314685314685]}

In [31]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(1, 0)

In [32]:

import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,"Alice went to a neat little house, on the doo...","alice, after following the white rabbit, finds..."
1,Alice was beginning to get tired of sitting b...,"alice, feeling tired from sitting with her sis..."


In [34]:

print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(1, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(1, "Reference_summaries"))

Actual Summary
 Alice was beginning to get tired of sitting by her sister on the bank, and of having nothing to do . She was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid) when suddenly a White Rabbit with pink eyes ran close by her . There was nothing so remarkable in that; nor did Alice think it so out of the way to hear the Rabbit say to itself, “Oh dear! Oh dear! I shall be late!”
Reference Summary
alice, feeling tired from sitting with her sister, sees a white rabbit who checks his pocket watch, sparking her curiosity. she follows the rabbit down a deep rabbit-hole that leads to a long fall through a well filled with books and maps. as she falls, she wonders if she'll reach the earth's center or go right through it. after the long fall, alice finds herself in a long hall with locked doors. she discovers a small door leading to a beautiful garden, but she is too large to fit through it. on a nearby table, she finds a bottle 

In [33]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [35]:
tokenizer(["four-time defending champion"])

{'input_ids': [[0, 10231, 12, 958, 5261, 2234, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [36]:

tokens = tokenizer.tokenize("four-time defending champion")

tokens

['four', '-', 'time', 'Ġdefending', 'Ġchampion']

In [37]:
ids = tokenizer.convert_tokens_to_ids(tokens)

ids

[10231, 12, 958, 5261, 2234]

In [38]:

from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [39]:

prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["summary"]]
    model_inputs = tokenizer(inputs, max_length = 1024, truncation = True)

    labels = tokenizer(text_target = examples["chosen"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [40]:

tokenized_ds = cleaned_ds.map(preprocess_function, batched = True)

tokenized_ds

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['book', 'chapter', 'chosen', 'summary', 'prompt', 'rejected', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7
    })
    test: Dataset({
        features: ['book', 'chapter', 'chosen', 'summary', 'prompt', 'rejected', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
})

In [41]:

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [42]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}

In [43]:

from huggingface_hub import notebook_login

notebook_login()

In [44]:



training_args = Seq2SeqTrainingArguments(
    output_dir = "cleaned_ds",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 3,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Generated Length
1,No log,4.505992,0.282600,0.038400,0.169400,0.169400,95.000000
2,No log,4.336765,0.283200,0.033300,0.170100,0.170100,82.000000
3,No log,4.280300,0.270500,0.036300,0.160900,0.160900,113.000000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=3, training_loss=4.058155377705892, metrics={'train_runtime': 429.9059, 'train_samples_per_second': 0.049, 'train_steps_per_second': 0.007, 'total_flos': 9015378591744.0, 'train_loss': 4.058155377705892, 'epoch': 3.0})

In [45]:

trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


events.out.tfevents.1727288906.12db4e28cce7.419.0:   0%|          | 0.00/8.16k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Musab-Bin-Nadeem/cleaned_ds/commit/8f529326ccb8b8ec8a8b27a214451113a5a96330', commit_message='End of training', commit_description='', oid='8f529326ccb8b8ec8a8b27a214451113a5a96330', pr_url=None, pr_revision=None, pr_num=None)